In [1]:
sc.install_pypi_package("boto3")
sc.install_pypi_package("requests")
sc.install_pypi_package("pandas==1.0.3")
sc.install_pypi_package("s3fs")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1638811140383_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import io
import pandas as pd
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import *
from pyspark.sql.functions import lit

s3client = boto3.client('s3')
resp = s3client.get_object(Bucket='political-movement-data', Key='key_terms.csv')
terms_df = pd.read_csv(resp.get("Body"))
movements = list(terms_df['Movement'])
pipeline = PretrainedPipeline('explain_document_dl', lang='en')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

explain_document_dl download started this may take some time.
Approx size to download 169.4 MB
[OK!]

In [58]:
import requests
import time

months = list(range(1,13))
yrs = list(range(2010,2021))
results = []
one_movement = ['Neoliberalism']
# Ended up doing one movement at at time because EMR kernel kept timing out but code is here to scale up
for movement in one_movement:
    term_string = str(list(terms_df[terms_df.Movement == movement]['Key Terms'])[0])
    key_terms = [x.strip() for x in term_string.split(',')]
    for yr in yrs:
        abstracts = []
        for month in months:
            nyt_url = 'https://api.nytimes.com/svc/archive/v1/' + str(yr) + '/' + str(month) + '.json?api-key=hTKZ4p0PT2r6M6T1K3P88fGF3xzZ3QDY'
            req = requests.get(nyt_url)
            res_json = req.json()
            docs = res_json['response']['docs']

            #count mentions
            p_cnt = 0
            for doc in docs:
                abstract = doc['abstract'].lower()
                snippet = doc['snippet'].lower()
                lead_para = doc['lead_paragraph'].lower()
                if (any(x in abstract for x in key_terms) or any(x in snippet for x in key_terms) > 0 or any(x in lead_para for x in key_terms)):
                    p_cnt += 1
                    abstracts.append([doc['abstract']])

            results.append([movement, p_cnt, month, yr])
            time.sleep(6)

    spark_abstracts = spark.createDataFrame(abstracts).toDF("text")
    spark_res = pipeline.transform(spark_abstracts)
    ner_res = spark_res.select(explode(arrays_zip('lemma.result',
                             'stem.result', 
                             'ner.result'
                             )).alias("cols")) \
    .select(expr("cols['0']").alias("lemma"),
          expr("cols['1']").alias("stem"),
          expr("cols['2']").alias("ner"),
         )
    persons = ner_res[(ner_res.ner == 'B-PER') | (ner_res.ner == 'I-PER')]
    per_cnt = persons[['lemma','ner']].groupBy('lemma').count()
    per_cnt = per_cnt.withColumn('Movement', lit(movement))
    per_cnt_pd = per_cnt.toPandas()

print(results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['Neoliberalism', 0, 1, 2010], ['Neoliberalism', 0, 2, 2010], ['Neoliberalism', 0, 3, 2010], ['Neoliberalism', 0, 4, 2010], ['Neoliberalism', 0, 5, 2010], ['Neoliberalism', 0, 6, 2010], ['Neoliberalism', 0, 7, 2010], ['Neoliberalism', 0, 8, 2010], ['Neoliberalism', 0, 9, 2010], ['Neoliberalism', 0, 10, 2010], ['Neoliberalism', 0, 11, 2010], ['Neoliberalism', 0, 12, 2010], ['Neoliberalism', 0, 1, 2011], ['Neoliberalism', 0, 2, 2011], ['Neoliberalism', 0, 3, 2011], ['Neoliberalism', 0, 4, 2011], ['Neoliberalism', 0, 5, 2011], ['Neoliberalism', 0, 6, 2011], ['Neoliberalism', 0, 7, 2011], ['Neoliberalism', 1, 8, 2011], ['Neoliberalism', 0, 9, 2011], ['Neoliberalism', 0, 10, 2011], ['Neoliberalism', 0, 11, 2011], ['Neoliberalism', 0, 12, 2011], ['Neoliberalism', 0, 1, 2012], ['Neoliberalism', 0, 2, 2012], ['Neoliberalism', 0, 3, 2012], ['Neoliberalism', 0, 4, 2012], ['Neoliberalism', 0, 5, 2012], ['Neoliberalism', 0, 6, 2012], ['Neoliberalism', 0, 7, 2012], ['Neoliberalism', 2, 8, 2012], [

In [59]:
df = pd.DataFrame(results, columns = ['Movement', 'Count','Month','Year'])

with io.StringIO() as csv_buffer:
    df.to_csv(csv_buffer, index=False)

    response = s3client.put_object(
        Bucket='political-movement-data', Key="output/NYT_Neoliberalism.csv", Body=csv_buffer.getvalue()
    )
    
with io.StringIO() as csv_buffer:
    per_cnt_pd.to_csv(csv_buffer, index=False)

    response = s3client.put_object(
        Bucket='political-movement-data', Key="output/NYT_Neoliberalism_NER.csv", Body=csv_buffer.getvalue()
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…